In [ ]:
! git clone https://github.com/ultralytics/yolov5 
! pip install -r yolov5/requirements.txt
! pip install gradio
! sudo apt install tesseract-ocr
! pip install pytesseract

In [ ]:
import pytesseract
import os
def make_rect(img,model):
  df=pd.DataFrame(columns=['Number_plate'])
  results = model(img)
  data=results.pandas().xyxy[0]
  new_img=img
  for i in range(len(data.index)):
    new_img=cv2.rectangle(new_img,(int(data.iloc[i].xmin),int(data.iloc[i].ymax)),(int(data.iloc[i].xmax),int(data.iloc[i].ymin)),(255,0,0),2)
    num_plt=new_img[int(data.iloc[i].ymin):int(data.iloc[i].ymax),int(data.iloc[i].xmin):int(data.iloc[i].xmax)]
    num_plt=cv2.cvtColor(num_plt,cv2.COLOR_BGR2GRAY)
    text=pytesseract.image_to_string(num_plt)
    text=text.replace('\n','')
    if text not in df.Number_plate.to_list() and text!= '\x0c':
      x={'Number_plate':text}
      df=df.append(x, ignore_index=True)
      if os.path.exists('/content/data.csv'):
        df1=pd.read_csv('/content/data.csv')
        df2=pd.concat([df1,df],ignore_index=True)
        df2.reset_index()
        os.remove('/content/data.csv')
      else:
        df2=df
      
      df2.to_csv('/content/data.csv',index=False)
    new_img=cv2.putText(new_img, text,[int(data.iloc[i].xmin),int(data.iloc[i].ymin)],cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
  return new_img

In [ ]:
import cv2
import torch
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
model = torch.hub.load('ultralytics/yolov5', 'custom' ,'/content/best.pt')
def func(vid_path):
  i=0
  vid=cv2.VideoCapture(vid_path)
  output = cv2.VideoWriter('/content/output.mp4', cv2.VideoWriter_fourcc(*'XVID'), 20, (int(vid.get(3)),int(vid.get(4))))
  while(vid.isOpened()):
    ret,frame=vid.read()
    
    if ret==False or i==100:
      break
    print(f"frame {i}")
    frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    new_frame=[]
    new_frame.append(make_rect(frame[0:int(np.shape(frame)[0]/2),0:int(np.shape(frame)[1]/2)],model))
    new_frame.append(make_rect(frame[0:int(np.shape(frame)[0]/2),int(np.shape(frame)[1]/2):int(np.shape(frame)[1])],model))
    new_frame.append(make_rect(frame[int(np.shape(frame)[0]/2):int(np.shape(frame)[0]),0:int(np.shape(frame)[1]/2)],model))
    new_frame.append(make_rect(frame[int(np.shape(frame)[0]/2):int(np.shape(frame)[0]),int(np.shape(frame)[1]/2):int(np.shape(frame)[1])],model))
    fin_frame=cv2.vconcat([cv2.hconcat([new_frame[0],new_frame[1]]),cv2.hconcat([new_frame[2],new_frame[3]])])
    fin_frame=cv2.cvtColor(fin_frame,cv2.COLOR_BGR2RGB)
    output.write(fin_frame)
    i+=1
  vid.release()
  output.release()
  print("vid released")
  return '/content/output.mp4'

In [ ]:
import gradio as gr
v=gr.Interface(func,gr.Video(),"playable_video")
v.launch(debug=True)